In [1]:
# Initiation
# importing required libraries
import os
import glob

import numpy as np
import pandas as pd
import sqlite3

In [2]:
path = os.sep+"home"+os.sep+"jupyter"+os.sep+"Team-Prophecy"
print(path)

/home/jupyter/Team-Prophecy


In [3]:
if path != os.getcwd():
    os.chdir(path)

In [4]:
from Code.src.modules.db_ops import *

In [5]:
process_path = os.path.join(os.sep+"home"+os.sep+"jupyter"+os.sep+"Team-Prophecy","Data","02_processed","intermediate.db")
print(process_path)

/home/jupyter/Team-Prophecy/Data/02_processed/intermediate.db


In [6]:
raw_path = os.path.join(os.sep+"home"+os.sep+"jupyter"+os.sep+"Team-Prophecy","Data","01_raw","CourseData","raw.db")
print(raw_path)

/home/jupyter/Team-Prophecy/Data/01_raw/CourseData/raw.db


In [7]:
raw_connection = sqlite3.connect(raw_path)

In [8]:
process_connection = sqlite3.connect(process_path)

In [9]:
r_student_details = pd.DataFrame(raw_connection.execute("SELECT * FROM student_details").fetchall(), columns=["stu_id","stu_admit_term_code","stu_deg_level","stu_college","stu_dept","stu_prog","stu_res","stu_visa","stu_bam"])

In [10]:
p_student_details = list(r_student_details.drop_duplicates().itertuples(index=False, name=None))

In [11]:
process_connection.executemany("""
INSERT INTO 
student_details(stu_id, stu_admit_term_code,
stu_deg_level, stu_college, stu_dept, 
stu_prog, stu_res, stu_visa, stu_bam) 
VALUES(?,?,?,?,?,?,?,?,?)
""",p_student_details)
process_connection.commit()

In [12]:
p_program = pd.DataFrame(process_connection.execute("SELECT prog_code FROM programs").fetchall(), columns=["prog_code"])
p_courses = pd.DataFrame(process_connection.execute("SELECT crs FROM courses").fetchall(), columns=["crs"])
p_program["temp"] = 0
p_courses["temp"] = 0
p_full_programcourses = p_program.merge(p_courses, on="temp", how="outer")
p_full_programcourses = p_full_programcourses.drop("temp",axis=1)
#r_program_course

In [13]:
r_program_course = pd.DataFrame(raw_connection.execute("SELECT prog_code, crs_id, required FROM program_course_offerings").fetchall(), columns=["prog_code","crs","required"])

In [14]:
p_programcourses = p_full_programcourses.merge(r_program_course,on=["prog_code","crs"],how="inner")

In [15]:
p_programcourses = list(p_programcourses.drop_duplicates().itertuples(index=False, name=None))

In [16]:
process_connection.executemany("""
    INSERT INTO program_course_offerings VALUES(?,?,?)
""",p_programcourses)
process_connection.commit()

In [17]:
r_course_section = pd.DataFrame(raw_connection.execute("SELECT * FROM course_section").fetchall(), columns=["crs","sect_id","sect_clg","sect_dept","sect_mod","sect_status","sect_lvl","sect_wiley"]).drop_duplicates()

In [18]:
colleges_camp = pd.DataFrame(process_connection.execute("SELECT col_name,campus_id_fk FROM colleges").fetchall(), columns=["sect_camp","sect_clg"]).drop_duplicates()

In [19]:
r_course_section["sect_clg"] = r_course_section["sect_clg"].str.encode('utf-8')
colleges_camp["sect_clg"] = colleges_camp["sect_clg"].str.encode('utf-8')

In [20]:
r_course_section = r_course_section.merge(colleges_camp, on=["sect_clg"])

In [21]:
r_course_section_list = list(r_course_section.itertuples(index=False, name=None))

In [22]:
process_connection.executemany("""
    INSERT INTO course_section(crs_id,
    sect_id,sect_clg,sect_dept,sect_camp,
    sect_mod,sect_status,sect_lvl,
    sect_wiley) 
    VALUES(?,?,?,?,?,?,?,?,?)
""",r_course_section_list)
process_connection.commit()